In [48]:
from modeling_gpt_neox import GPTNeoXForCausalLM
model = GPTNeoXForCausalLM.from_pretrained("trl-internal-testing/tiny-random-GPTNeoXForCausalLM")
from modeling_gpt_neox import GPTNeoXAttention
attention = GPTNeoXAttention(config=model.config)

In [49]:
new_state_dict = {}
for k, v in model.state_dict().items():
    if k.startswith("gpt_neox.layers.1.attention."):
        new_state_dict[k.replace("gpt_neox.layers.1.attention.", "")] = v

attention.load_state_dict(new_state_dict)

<All keys matched successfully>

In [66]:
import torch
inputState = torch.randn(1, 3, model.config.hidden_size)

attention(inputState)

tensor([[[-5.8310e-02, -1.6087e-01, -2.0428e-01, -2.4702e-02, -2.0972e-01,
           8.7988e-02, -2.6497e-01,  2.7737e-02,  1.2506e-01, -2.6803e-02,
          -1.1552e-02,  4.4691e-02, -3.9390e-02, -2.2193e-02,  3.2879e-02,
          -5.4363e-02, -8.1543e-02, -1.6463e-01,  1.6081e-02,  3.3943e-02,
           4.6513e-03,  2.4262e-02, -3.7914e-02, -2.5462e-01,  1.0213e-01,
           1.4505e-01, -4.7030e-02, -1.1044e-01,  3.2030e-04,  6.8774e-02,
           7.7158e-03,  3.0727e-02, -1.7373e-02, -1.5557e-01, -2.0885e-01,
           7.3585e-02, -3.4552e-02,  1.4937e-01, -1.7941e-01,  2.3737e-01,
           6.8656e-02, -2.1996e-01, -1.1606e-01, -1.0428e-01, -5.2216e-02,
          -1.0184e-01, -2.4314e-02, -1.8734e-01,  4.6922e-02,  2.3626e-01,
           4.5409e-02,  6.1575e-02,  5.7560e-02, -5.6481e-03, -1.2850e-01,
           3.5208e-02,  7.2847e-02,  2.0259e-01, -7.0605e-02, -1.2796e-01,
          -3.6485e-02,  4.1885e-02, -8.9488e-03, -1.9154e-01,  3.6256e-02,
          -2.2605e-02,  4

(tensor([[[ 8.4380e-04, -1.8707e-02, -7.9106e-03, -2.1717e-02, -1.4750e-04,
            1.0221e-02,  9.0379e-03, -1.3679e-02,  3.0044e-03, -3.0853e-03,
            1.9691e-03, -5.2051e-03, -4.7137e-03, -1.4293e-03, -1.5147e-03,
            2.3709e-02,  5.7025e-03, -1.2336e-02, -3.1603e-03, -2.0023e-02,
            2.8193e-02, -1.0411e-02, -1.4372e-02,  3.0220e-02, -8.8097e-03,
           -1.3184e-03, -9.6720e-03,  3.7320e-03,  2.0063e-02, -5.3158e-03,
           -2.9216e-03,  6.4100e-03],
          [ 2.2677e-03, -1.3946e-02, -3.2214e-03, -1.9329e-02,  2.7390e-03,
            8.7879e-03,  1.1250e-02, -4.1818e-03,  1.1194e-02, -8.2719e-04,
            4.4894e-03, -4.9629e-03,  5.1314e-06,  2.4338e-03,  2.5025e-03,
            2.6363e-02,  1.2752e-02, -1.2245e-02, -1.6854e-03, -1.1198e-02,
            2.1714e-02, -1.0845e-02, -8.9126e-03,  2.0203e-02, -1.3107e-02,
           -1.7376e-03,  1.5407e-03,  6.5900e-03,  2.0691e-02, -1.1573e-02,
           -5.4017e-03,  7.8119e-03],
          [ 

In [67]:
print(model.config.hidden_size)
print(model.config.num_attention_heads)
print(model.config.max_position_embeddings)
print(model.config.hidden_size // model.config.num_attention_heads)

32
4
512
8


In [68]:
from torch.nn import functional as F
import math
import torch
import torch.nn as nn
class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.hidden_size % config.num_attention_heads == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.hidden_size, 3 * config.hidden_size, bias=True)#config.bias)
        # output projection
        self.c_proj = nn.Linear(config.hidden_size, config.hidden_size, bias=True) #n_embd = hidden_size?
        # regularization
        self.attn_dropout = nn.Dropout(0.0)
        self.resid_dropout = nn.Dropout(0.0)
        self.n_head = config.num_attention_heads #config.n_head
        self.n_embd = config.hidden_size
        self.dropout = 0.0
        # flash attention make GPU go brrrrr but support is only in PyTorch nightly and still a bit scary
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention') and self.dropout == 0.0
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention atm needs PyTorch nightly and dropout=0.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.max_position_embeddings, config.max_position_embeddings))
                                        .view(1, 1, config.max_position_embeddings, config.max_position_embeddings)) #block_size? -> am I using this wrong?

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)
        head_size = C // self.n_head
        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        qkv = self.c_attn(x)
        # print(qkv.size(), qkv[2][5][1]) #SAME
        
        # TODO: Alexs addition -> works for 1,1 -> now figure out how to make it work for BIGGER
        q, k ,v  = qkv.view(4, -1, 8).transpose(0, 1)#qkv.split(self.n_embd, dim=2)
        # print('q1', q)
        # print('k', k)
        # # print(q.shape, k.shape, v.shape)
        k = k.view(B, T, self.n_head, head_size).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, head_size).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, head_size).transpose(1, 2) # (B, nh, T, hs)
        # print('k', k)
        # print('qkv1', qkv)

        # <NEW>
        # new_qkv_shape = qkv.size()[:-1] + (self.n_head, 3 * head_size)
        # print(new_qkv_shape)
        # qkv = qkv.view(*new_qkv_shape)
        # print('qkv2', qkv)
        # [batch, seq_len, num_attention_heads, 3 * head_size] --> 3 [batch, num_attention_heads, seq_len, head_size]
        # q = qkv[..., : head_size].permute(0, 2, 1, 3)
        # k = qkv[..., head_size : 2 * head_size].permute(0, 2, 1, 3)
        # v = qkv[..., 2 * head_size :].permute(0, 2, 1, 3)
        # print('q2', q)
        # </NEW>
        
        # print(q.shape, k.shape, v.shape)
        # print(q[0][0][0][1], k[0][0][0][1], v[0][0][0][1])
        # print(q[0][0][0][0], k[0][0][0][0], v[0][0][0][0])

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # print(y.shape)
        # output projection
        # print(y[0][0][0])
        y = self.resid_dropout(self.c_proj(y))
        return y

attention2 = CausalSelfAttention(model.config)
new_state_dict = {
    "c_proj.weight": model.state_dict()["gpt_neox.layers.1.attention.dense.weight"],
    "c_proj.bias": model.state_dict()["gpt_neox.layers.1.attention.dense.bias"],
    "c_attn.weight": model.state_dict()["gpt_neox.layers.1.attention.query_key_value.weight"],
    "c_attn.bias": model.state_dict()["gpt_neox.layers.1.attention.query_key_value.bias"],
    "bias": model.state_dict()["gpt_neox.layers.1.attention.bias"],
}
# print([(k, v.size()) for k,v in attention2.state_dict().items()])
# print([(k, v.size()) for k,v in new_state_dict.items()])

attention2.load_state_dict(new_state_dict)

from modeling_gpt import GPT2Attention
gpt2Attention = GPT2Attention(model.config)

new_state_dict['c_attn.weight'] = new_state_dict['c_attn.weight'].t()
new_state_dict['c_proj.weight'] = new_state_dict['c_proj.weight'].t()
new_state_dict['masked_bias'] = gpt2Attention.state_dict()['masked_bias']
gpt2Attention.load_state_dict(new_state_dict)


x = attention2(inputState)
y = attention(inputState)[0]
torch.allclose(x,y)
# z = gpt2Attention(inputState)[0]

ValueError: too many values to unpack (expected 3)

In [53]:

x[0][0][0], y[0][0][0]#, z[0][0][0]

(tensor(-0.0068, grad_fn=<SelectBackward0>),
 tensor(-0.0068, grad_fn=<SelectBackward0>))

In [54]:
torch.allclose(x,y)

True

# SUCCESS
It worked, we figured it out!

So, the *two* differneces between the self attentions are:
1. How they each unpack the `qkv`'s
2. Addition of rotary embeddings

It is *completely* unclear to me why they each do these different variations for the unpacking. If I'm lucky I might be able to figure out how to reshape things to get the same results.

I think that's the next test I should do in that case... isolate out each of these pieces and try to decipher how they manipulate the various weights
If I can end up minimising the difference between them, then *fuck yes*.
